In [1]:
from nltk.tokenize import TweetTokenizer
from gensim import corpora, models, matutils
import numpy as np
from sklearn import metrics
from operator import itemgetter
from collections import Counter
import json
import gzip
from pprint import pprint
import os

In [2]:
def read_tweets_from_file(category, results_path):
    switcher = {
        "nw": "nw.txt",
        "food": "food.txt",
        "politics": "politics.txt",
        "wellness": "wellness.txt",
        "entertainment": "entertainment.txt",
        "election": "election.txt",
        "nasa":"nasa.txt",
        "addele" : "addele.txt",
        "sport" : "sport.txt"
        }
    file_name = switcher.get(category)
    with open(results_path + file_name, 'rb') as f:
        tweets = f.read().splitlines()
    f.close()
    return tweets

In [95]:
def Clustering_Tweets(category, path, cluster_size):
    
    tweets = read_tweets_from_file(category, path)
    #print len(tweets_all)
    #tweets = set(tweets_all)
    #print len(tweets)
    
    stoplist = set([line.replace("\n","") for line in open('/Users/ynh3/Data/stopword-list2.txt')])
    signallist = ["@reuters", "@ajam","@ajenglish","@yahoonews","@nytimes","@nyt","@npr","@nprnews","@breakingnews","@bbcbreaking","@bbc","@cnnbrk","@cnn","@cnni"]
    pun = [',', '-','.', ':', '(', ')', '--', ';', '...', 'today','follow','stats','can', 'say', 'says', 'will','may', 'must', 'us', 'via','a','the', 'rt', 'gg', 'gt', 'lt', 'la', 'de', 'te', 'lol', 'follow', 'followers', 'unfollow', 'unfollowers', 'unfollower', 'follower']

    common = [('love', 35339), ('today', 24001), ('con', 22008), ('day', 18141), ('people', 17123), ('time', 16630), ('video', 16419), ('life', 11496), ('est', 11168), ('thanks', 10098), ('com', 9479), ('post', 8939), ('shit', 8923), ('todo', 8345), ('stats', 7991), ('work', 7969), ('photo', 7684), ('world', 7648), ('way', 7358), ('son', 6878), ('birthday', 6837), ('person', 6711), ('night', 6647), ('girl', 6595), ('vida', 6548), ('game', 6284), ('home', 6170), ('update', 6091), ('week', 5496), ('baby', 5311), ('show', 5239), ('days', 5209), ('year', 5162), ('morning', 4947), ('phone', 4796), ('music', 4754), ('school', 4720), ('dm', 4676), ('ass', 4675), ('news', 4660), ('jam', 4552), ('guys', 4487), ('friends', 4486), ('sleep', 4454), ('todos', 4416), ('os', 4388), ('emojis', 4363), ('tak', 4353), ('tomorrow', 4254)]
    common_word = [e[0] for e in common]
    common_word = common_word + ['like','latest','who']
    #len(common_word)

    #Tokenize
    tknzr = TweetTokenizer()   
    tweet_count = 0
    tweet_tokens=[]
    tweet_list=[]
    i = 0

    for line in tweets:

        new_l = tknzr.tokenize(line)
        lower_l = [e.lower() for e in new_l]
        new_ll = [e for e in lower_l if e not in stoplist and e not in common_word and e not in pun and len(e) > 2 and e.isalpha() and 'http' not in e and '@' not in e]
        #if (new_ll not in tweet_tokens):
        if len(new_ll) > 0:
            tweet_tokens.append(new_ll)
            tweet_count += 1
            tweet_list.append(line)
        i += 1

    print "Done with tokenizations. tweets count: " , tweet_count
    
    dictionary = corpora.Dictionary(tweet_tokens)
    print(dictionary)

    corpus = [dictionary.doc2bow(t) for t in tweet_tokens]

    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]

    n_cluster = cluster_size
    
    # LSI computing
    lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=n_cluster) # initialize an LSI transformation
    corpus_lsi = lsi[corpus]

    j = 0
    lsi_topic_labels=[]
    for i,doc in enumerate(lsi[corpus_tfidf]):
        a = np.array(doc)
        absA= abs(a)
        if (len(a)==0):
            cluster_id=0
            prob_value=0.0
            j=j+1
        else:
            try:
                topicmax = absA.argmax(axis=0)
            except ValueError:
                print "ValueError at"
                print i
            cluster_id = a[topicmax.item(1)][0]
            prob_value = abs(a[topicmax.item(1)][1])

        triple = i, cluster_id, prob_value
        lsi_topic_labels.append(triple)
        l = lsi.print_topic(cluster_id,topn=10)
        prob_list = [(abs(float(a[:5])),a[6:])  for a in (l.split('+ '))]
        sorted(prob_list)
        t_words = [w[1] for w in prob_list]
        i=i+1

    '''topics = []
    for i in range(0, n_cluster):
        l = lsi.print_topic(i,topn=5)
        prob_list = [(abs(float(a[:5])),a[6:])  for a in (l.split('+ ')) if abs(float(a[:5])) > 0.3]
        sorted(prob_list)
        topics.append(prob_list)'''
    
    ##For silhouette score
    labels = [b for a,b,c in lsi_topic_labels]
    np_labels = np.asarray(labels)

    # convert corpus to array
    corpusAsMatrix = matutils.corpus2dense(corpus_lsi, num_terms=500).transpose()

    score = metrics.silhouette_score(corpusAsMatrix, np_labels, metric='cosine')

    print 'silhouette score is: {}'.format(score)
    
    ###MAKING READY FOR
    temp = sorted(lsi_topic_labels,key=itemgetter(1,2), reverse=True)
    cluster_topic_list = [e for e in temp if float(e[2]) > 0.2]
    leftovers_list = [tweet_list[e[0]] for e in temp if float(e[2]) <= 0.2]
    #Counter(a,b,c for a,b,c in lsi_topic_labels)

    cluster_list = sorted(cluster_topic_list,key=lambda x: x[1])
    counter = Counter(b for a,b,c in cluster_list)
    counter_sort = counter.most_common(n_cluster)
    cid_sort = [e[0] for e in counter_sort]

    print 'length of lsi_topic_labels is {}'.format(len(lsi_topic_labels))
    print 'length of cluster_list is {}'.format(len(cluster_list))
    print 'length of counter_sort is {}'.format(len(counter_sort))
    print 'n_cluster is {}'.format(n_cluster)
    print 'cid_sort is {}'.format(len(cid_sort))

    t = lsi.show_topics()
    #with open(path + category + '_exp1_topics_v1_' + str(cluster_size) + '.txt', 'wt') as myfile:
    #    myfile.write('\n\n'.join(t))
    myfile = open(path + category + '_exp1_topics_v1_' + str(cluster_size) + '.txt', 'wt')
    
        
    #def myfunction():
    output_file = open(path + category + '_exp1_clusters_v1_' + str(cluster_size) + '.txt', 'w')

    data_list=[]
    data = {}
    count=0
    i=0

    for i in range(0, len(cid_sort)):
        sublist = [e for e in cluster_topic_list if e[1] == cid_sort[i]]
        #print "index", t_index
        t_index = int(cid_sort[i])
        #count = int(counter_sort[i][1])
        #start = sum([e[1] for e in counter_sort[:i]])
        #sublist = cluster_topic_list[start:count]
        #pdb.set_trace()
        #if i < 2: print start, count, len(sublist)

        topic = lsi.print_topic(t_index,topn=10)
        prob_list = [((float(a[:5])),a[6:])  for a in (topic.split('+'))]

        neg_count = [w[0] for w in prob_list if w[0] < 0]
        pos_count = [w[0] for w in prob_list if w[0] > 0]
        #print neg_count, pos_count

        if len(pos_count) > len(neg_count):
            t_words = [w[1] for w in prob_list if w[0] > 0]
        else:
            t_words = [w[1] for w in prob_list if w[0] < 0]

        topic = ''.join(t_words)
        
        #topic = topics[t_index]
        #string = [x[1] for x in topic]
        string = ' '.join(topic)
        #print topic
        if string == "":
            print i
            print cid_sort[i], t_index
            print topic
            #print sublist

            break
 
        l = [(float(ss[2]),tweet_list[ss[0]]) for ss in sublist if float(ss[2]) > 0.2]
        #print len(l_all)
        #l = set(l_all)
        #print len(l)

        myfile.write(topic +'\n')
        
        output_file.write("\n\n")
        output_file.write('Cluster # %d' %t_index + '-----------------------\n')
        output_file.write('Total items # %d' %len(l) + '-----------------------\n\n')
        output_file.write('Topic: %s' %topic +'\n')

        print('Cluster # %d' %t_index + '-----------------------\n')
        print('Total items # %d' %len(l) + '-----------------------\n\n')
        print('Topic: %s' %topic+'\n')
        print "".join("%s, %s\n" % tup for tup in l)
        #print("\n".join(l))

        #output_file.write("\n".join(l))
        output_file.write("".join("%s, %s\n" % tup for tup in l))

    #print("Total number of tweets is {}".format(count))
    output_file.close()


In [96]:
Clustering_Tweets("nw","/Users/ynh3/Documents/ExS/results/categorized_tweets_enhanced_5thweek/",5)

Done with tokenizations. tweets count:  9579
Dictionary(8179 unique tokens: [u'foul', u'jihad', u'conjuring', u'payoff', u'looking']...)
silhouette score is: 0.213668331504
length of lsi_topic_labels is 9579
length of cluster_list is 1213
length of counter_sort is 5
n_cluster is 5
cid_sort is 5
Cluster # 0-----------------------

Total items # 531-----------------------


Topic: *"nepal" 1*"baltimore" 2*"earthquake" 3*"death" 2*"police" 4*"toll" 

0.67346986479, @cnngermanypr nepal earthquake http://t.co/p3ik6lkas3
0.578242424863, rt @cnnbrk: death toll from nepal earthquake rises to more than 4,700, with more than 9,000 injured. http://t.co/nawhlfpqs5.
0.578242424863, rt @cnnbrk: death toll from nepal earthquake rises to more than 4,700, with more than 9,000 injured. http://t.co/nawhlfpqs5.
0.578242424863, rt @cnnbrk: death toll from nepal earthquake rises to more than 4,700, with more than 9,000 injured. http://t.co/nawhlfpqs5.
0.578242424863, rt @cnnbrk: death toll from nepal earthq

In [ ]:
#######################

In [29]:
myList = [(1,2),(3,4)]
print "".join("%s %s\n"  %tup for tup in myList)

1 2
3 4

